In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
from torch.autograd import Variable
from sklearn.preprocessing import MinMaxScaler

In [2]:
# Training Data parameters
seq_len = 10
hidden_size = 25 # hidden features
num_layers = 1 # Default is 1, 2 is a stacked LSTM
output_dim = 1 # x,y,z
input_size = 1
# Training parameters
num_epochs = 2000
learning_rate = 0.01

In [3]:
data = np.arange(2000)

In [4]:
def sliding_windows(data, seq_length):
    x = []
    y = []

    for i in range(len(data)-seq_length-1):
        _x = data[i:(i+seq_length)]
        _y = data[i+seq_length]
        x.append(_x)
        y.append(_y)

    return np.array(x),np.array(y)

In [10]:
scale = False
if scale:
    sc = MinMaxScaler()
    data_sc = sc.fit_transform(data.reshape(-1,1))
else:
    data_sc = data.reshape(-1,1)
x,y = sliding_windows(data_sc,seq_len)

train_size = int(len(y) * 0.67)
test_size = len(y) - train_size

dataX = Variable(torch.Tensor(np.array(x)))
dataY = Variable(torch.Tensor(np.array(y)))

trainX = Variable(torch.Tensor(np.array(x[0:train_size])))
trainY = Variable(torch.Tensor(np.array(y[0:train_size])))

testX = Variable(torch.Tensor(np.array(x[train_size:len(x)])))
testY = Variable(torch.Tensor(np.array(y[train_size:len(y)])))

In [11]:
trainX.shape

torch.Size([1332, 10, 1])

In [12]:
class LSTM(nn.Module):

    def __init__(self, output_dim, input_size, hidden_size, num_layers):
        super(LSTM, self).__init__()
        
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.seq_len = seq_len
        
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size,
                            num_layers=num_layers, batch_first=True)
        
        self.fc = nn.Linear(hidden_size, output_dim)

    def forward(self, x):
        h_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        c_0 = Variable(torch.zeros(
            self.num_layers, x.size(0), self.hidden_size))
        
        # Propagate input through LSTM
        ula, (h_out, _) = self.lstm(x, (h_0, c_0))
        
        h_out = h_out.view(-1, self.hidden_size)
        
        out = self.fc(h_out)
        
        return out

In [13]:
lstm = LSTM(output_dim, input_size, hidden_size, num_layers)

criterion = torch.nn.MSELoss()    # mean-squared error for regression
optimizer = torch.optim.Adam(lstm.parameters(), lr=learning_rate)

# Train the model
for epoch in range(num_epochs):
    outputs = lstm(trainX)
    optimizer.zero_grad()
    
    # obtain the loss function
    loss = criterion(outputs, trainY)
    
    loss.backward()
    
    optimizer.step()
    if epoch % 100 == 0:
      print("Epoch: %d, loss: %1.8f" % (epoch, loss.item()))

Epoch: 0, loss: 604223.87500000
Epoch: 100, loss: 574758.56250000
Epoch: 200, loss: 548230.00000000
Epoch: 300, loss: 519019.78125000
Epoch: 400, loss: 488685.75000000
Epoch: 500, loss: 462032.81250000
Epoch: 600, loss: 434171.87500000
Epoch: 700, loss: 409289.71875000
Epoch: 800, loss: 386318.06250000
Epoch: 900, loss: 364839.43750000
Epoch: 1000, loss: 344648.96875000
Epoch: 1100, loss: 325623.21875000
Epoch: 1200, loss: 307641.96875000
Epoch: 1300, loss: 290633.43750000
Epoch: 1400, loss: 274531.65625000
Epoch: 1500, loss: 259278.25000000
Epoch: 1600, loss: 244821.26562500
Epoch: 1700, loss: 231115.34375000
Epoch: 1800, loss: 218115.43750000
Epoch: 1900, loss: 205785.56250000


In [14]:
lstm(trainX[:20])

tensor([[ 9.9840],
        [10.9622],
        [12.0166],
        [13.0350],
        [14.0195],
        [14.9492],
        [15.9105],
        [16.9601],
        [18.0294],
        [19.0341],
        [19.9849],
        [20.9485],
        [21.9611],
        [23.0048],
        [24.0424],
        [25.0528],
        [26.0384],
        [27.0118],
        [27.9849],
        [28.9639]], grad_fn=<AddmmBackward>)

In [15]:
trainY[:20]

tensor([[10.],
        [11.],
        [12.],
        [13.],
        [14.],
        [15.],
        [16.],
        [17.],
        [18.],
        [19.],
        [20.],
        [21.],
        [22.],
        [23.],
        [24.],
        [25.],
        [26.],
        [27.],
        [28.],
        [29.]])